In [9]:
import os
import shutil

# Ordnerpfade definieren
google_drive_dir = '/Users/MeinNotebook/DocDig/Scans_for_Dataset/Vogelarten'
additional_data_dir = '/Users/MeinNotebook/DocDig/Scans_for_Dataset/Dataset'
combined_dir = '/Users/MeinNotebook/DocDig/Scans_for_Dataset/CombinedDataset'

# Zielordner neu erstellen
os.makedirs(combined_dir, exist_ok=True)

# Alle Klassen aus dem Google-Drive-Ordner übernehmen
for class_name in os.listdir(google_drive_dir):
    # Überspringe Ordner mit Anführungszeichen im Namen
    if '"' in class_name or "'" in class_name:
        print(f"⚠️  Übersprungen (ungültiger Ordnername): {class_name}")
        continue

    src_gd_class = os.path.join(google_drive_dir, class_name)
    src_additional_class = os.path.join(additional_data_dir, class_name)
    dst_class = os.path.join(combined_dir, class_name)

    if not os.path.isdir(src_gd_class):
        continue

    os.makedirs(dst_class, exist_ok=True)

    # Bilder aus Google Drive kopieren
    for file in os.listdir(src_gd_class):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            shutil.copy2(os.path.join(src_gd_class, file), os.path.join(dst_class, file))

    # Falls diese Klasse auch im Zusatzordner vorhanden ist: Bilder ebenfalls kopieren
    if os.path.isdir(src_additional_class):
        for file in os.listdir(src_additional_class):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                target_path = os.path.join(dst_class, file)
                if not os.path.exists(target_path):  # nur kopieren, wenn noch nicht vorhanden
                    shutil.copy2(os.path.join(src_additional_class, file), target_path)

print(f"✅ Kombinierter Datensatz wurde unter '{combined_dir}' erstellt.")



⚠️  Übersprungen (ungültiger Ordnername): "
✅ Kombinierter Datensatz wurde unter '/Users/MeinNotebook/DocDig/Scans_for_Dataset/CombinedDataset' erstellt.


In [2]:
# Train of mnsist cnn

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 110ms/step - accuracy: 0.8474 - loss: 0.4842 - val_accuracy: 0.9840 - val_loss: 0.0511
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 55s 117ms/step - accuracy: 0.9734 - loss: 0.0923 - val_accuracy: 0.9875 - val_loss: 0.0371
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 110ms/step - accuracy: 0.9820 - loss: 0.0619 - val_accuracy: 0.9901 - val_loss: 0.0313
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 110ms/step - accuracy: 0.9841 - loss: 0.0526 - val_accuracy: 0.9894 - val_loss: 0.0330
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 52s 111ms/step - accuracy: 0.9857 - loss: 0.0476 - val_accuracy: 0.9911 - val_loss: 0.0313
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 55s 117ms/step - accuracy: 0.9877 - loss: 0.0395 - val_accuracy: 0.9909 - val_loss: 0.0302
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 56s 119ms/step - accuracy: 0.9885 - loss: 0.0358 - val_accuracy: 0.9914 - val_loss: 0.0273
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 106ms/step - accuracy: 0.9888 - loss: 0

In [ ]:
model.save('mnist_cnn_model.keras')

In [10]:
#train of species cnn

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import json

# Pfad zu Ihrem Hauptordner
base_dir = '/Users/MeinNotebook/DocDig/Scans_for_Dataset/CombinedDataset'

# Bildgrößen und Batchgröße definieren
img_height, img_width = 22, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

# Keine Datenanreicherung für Validierungsdaten
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# CNN Modell erstellen

model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D(2, 2),
    Dropout(0.1),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D(2, 2),
    Dropout(0.1),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(len(train_generator.class_indices), activation='softmax')
])


class_indices = train_generator.class_indices
with open('/Users/MeinNotebook/DocDig/class_indices.json', 'w') as file:
    json.dump(class_indices, file)
    
    
    
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks definieren
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('vogelarten_best_model.keras', save_best_only=True)

# Modell trainieren
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,  # Anzahl der Epochen kann angepasst werden
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Found 4429 images belonging to 28 classes.
Found 1097 images belonging to 28 classes.
Epoch 1/50


/Users/MeinNotebook/DocDig/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.1702 - loss: 3.8262 - val_accuracy: 0.3199 - val_loss: 2.5839
Epoch 2/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 186us/step - accuracy: 0.1875 - loss: 2.5217 - val_accuracy: 0.3333 - val_loss: 2.5219
Epoch 3/50
  7/138 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3719 - loss: 2.3350

/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.3600 - loss: 2.3733 - val_accuracy: 0.4338 - val_loss: 2.2741
Epoch 4/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 145us/step - accuracy: 0.4375 - loss: 2.3457 - val_accuracy: 0.5556 - val_loss: 2.0480
Epoch 5/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4727 - loss: 1.9744 - val_accuracy: 0.5009 - val_loss: 2.0910
Epoch 6/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 158us/step - accuracy: 0.6562 - loss: 1.8190 - val_accuracy: 0.5556 - val_loss: 1.3372
Epoch 7/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5378 - loss: 1.8168 - val_accuracy: 0.5202 - val_loss: 2.0291
Epoch 8/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 65us/step - accuracy: 0.4688 - loss: 1.9045 - val_accuracy: 0.5556 - val_loss: 2.8931
Epoch 9/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5780 - loss: 1.7058 - val_accuracy: 0.5579 - val_loss: 1.9451
Epoch 10/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 149us/step - accuracy: 0.5938 - loss: 1.5622 - val_accuracy: 0

2025-05-04 12:46:15.669583: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6748 - loss: 1.4454 - val_accuracy: 0.5956 - val_loss: 1.8101
Epoch 18/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 64us/step - accuracy: 0.5312 - loss: 1.9003 - val_accuracy: 0.3333 - val_loss: 1.8974
Epoch 19/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6981 - loss: 1.3911 - val_accuracy: 0.6158 - val_loss: 1.7634
Epoch 20/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 65us/step - accuracy: 0.7500 - loss: 1.3273 - val_accuracy: 0.6667 - val_loss: 1.2287


In [11]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Hole die wahren Labels und Vorhersagen aus dem Validation-Generator
# Achtung: shuffle=False wichtig für korrekte Zuordnung!
validation_generator.reset()
y_true = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Vorhersagen berechnen
Y_pred = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

# Klassifikationsbericht anzeigen
print("📊 Klassengenauigkeit:")
report = classification_report(y_true, y_pred, target_names=class_labels, digits=2)
print(report)


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
📊 Klassengenauigkeit:
                    precision    recall  f1-score   support

                 "       0.00      0.00      0.00        20
        Alpenmeise       0.00      0.00      0.00        21
        Baumpieper       0.00      0.00      0.00        24
      Birkenzeisig       0.04      0.05      0.05        42
         Blaumeise       0.08      0.11      0.09       136
      Braunkelchen       0.00      0.00      0.00        20
        Distelfink       0.07      0.05      0.06        43
       Erlenzeisig       0.07      0.05      0.06        86
             Fitis       0.17      0.05      0.07        21
  Gartenrotschwanz       0.00      0.00      0.00        20
       Gartenrötel       0.00      0.00      0.00        30
        Gelbspöter       0.12      0.15      0.14        20
           Girlitz       0.09      0.11      0.10        46
     Grauschnäpper       0.00      0.00      0.00        23
    Hausrotschwanz       0.05      0.

In [5]:
model.save('vogelarten_final_model.keras')